## Setup

In [ ]:
import os
import cv2
import numpy as np
from keras.utils import image_dataset_from_directory
import tensorflow as tf
import matplotlib.pyplot as plt
import gdown
from keras.models import load_model
from tqdm.notebook import tqdm
from deepface import DeepFace

In [ ]:
model = DeepFace.build_model("DeepFace")
model.model = tf.keras.models.load_model("deepface_model.h5")

In [ ]:
test_ds = image_dataset_from_directory(
    "test_aligned",
    labels="inferred",
    label_mode="int",
    image_size=(152, 152),
)

In [ ]:
# we can move it to utils file

def load_database(path):
    dataset = {}
    for person in os.listdir(path):
        person_name = person.split(".")[0]
        dataset[person_name] = np.load(os.path.join(path, person))
    return dataset

def get_average_embedding(path):
    total_embeddings = None
    for img in os.listdir(path):
        image = cv2.imread(os.path.join(path, img))
        image = np.expand_dims(image, axis=0)
        try:
            detected_face_embedding = model.find_embeddings(image)
        except:
            continue
        if total_embeddings is None:
            total_embeddings = detected_face_embedding
        else:
            total_embeddings = np.vstack((total_embeddings, detected_face_embedding))
    return np.mean(total_embeddings, axis=0)

In [ ]:
test_dataset = load_database("test_database")

## Luminance

In [ ]:
def scale_luminance(image, method, factor=None):
    image_yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    y = image_yuv[:,:,0]

    if method == "square":
      y = np.clip(y ** 2, 0, 255).astype(np.uint8)
    if method == "linear":
      y = np.clip(y * factor, 0, 255).astype(np.uint8)
    if method == "const":
      y = np.clip(y + factor, 0, 255).astype(np.uint8)

    image_yuv[:,:,0] = y
    image = cv2.cvtColor(image_yuv, cv2.COLOR_YUV2RGB)

    return image


### Test

In [ ]:
test_paths = os.listdir("test_aligned")
test_paths = [os.path.join("test_aligned", path) for path in test_paths if not path.startswith(".")]

In [ ]:
test_image = cv2.imread(os.path.join(test_paths[0], os.listdir(test_paths[0])[0]))

In [ ]:
fig, axes = plt.subplots(1, 6, figsize=(14, 10))

axes[0].imshow(test_image)
axes[1].imshow(scale_luminance(test_image, "square"))
axes[2].imshow(scale_luminance(test_image, "linear", 0.5))
axes[3].imshow(scale_luminance(test_image, "linear", 1.5))
axes[4].imshow(scale_luminance(test_image, "const", -100))
axes[5].imshow(scale_luminance(test_image, "const", 30))

plt.tight_layout()
plt.show()

### Experiment

In [ ]:
PARAMS = [("linear", 1/2), ("linear", 3/5), ("linear", 3/4), ("linear", 4/3), ("linear", 3/2),
          ("const", -100), ("const", -20), ("const", -10), ("const", 30)]

correct = 0
total = 0

for images, labels in test_ds:
    for img, label in zip(images, labels):
        for param in PARAMS:
          img_to_model = scale_luminance(img.numpy(), param)
          img_to_model = tf.expand_dims(img_to_model, axis=0)
          try:
              detected_face_embedding = model.find_embeddings(img_to_model)
          except:
              continue
          distances = []
          for person in test_dataset:
              distances.append(np.linalg.norm(detected_face_embedding - test_dataset[person]))
          if np.argmin(distances) == label:
              correct += 1
          total += 1

correct/total